# Week 3 --- Explore Toronto Neighbourhood
*Author: Zinan LIN*

*Note: some code are from course tutorial: https://labs.cognitiveclass.ai/tools/jupyterlab/lab/tree/labs/DP0701EN/DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb*


## Part 1: Web Scrapper

import libraries

In [54]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

create web page object handler

In [55]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'html5lib')

locate the table and split the rows

In [56]:
web_table = soup.table.find_all('td')

table = [i.text.rstrip() for i in web_table]
raw_postal = table[::3]
raw_borough = table[1::3]
raw_neigh = table[2::3]

create a dataframe and populate the dataframe with data in the table

In [57]:
toronto_post = pd.DataFrame(columns=['PostalCode','Borough','Neighbourhood'])

for i in range(len(raw_postal)):
    
    # if borough undefined then ignore
    if raw_borough[i] == 'Not assigned':
        continue
    
    # if more than one neighborhood exists in one postal code area, combine them
    if raw_postal[i] in toronto_post['PostalCode'].tolist():
        
        # if neighborhood undefined but borough is then neighborhood = borough
        tempNei = raw_borough[i] if raw_neigh[i] == 'Not assigned' else raw_neigh[i]
        
        # find the index of the duplicated postal
        index = toronto_post['PostalCode'].tolist().index(raw_postal[i])
        
        # combine them with comma seperated
        toronto_post.at[index, 'Neighbourhood'] += ', ' + tempNei
        continue
    
    # new entry, append to the dataframe
    toronto_post = toronto_post.append({'PostalCode':raw_postal[i], 
                                        'Borough':raw_borough[i], 
                                        'Neighbourhood':raw_neigh[i]},
                                       ignore_index=True)


__Here are some assumptions about the above data processing steps__ :
- If a postal code is duplicted, then the borough also is. (since the underlying meaning of postal code is to seperate boroughs
- The indexing of the dataframe is based on the order/ indexing of raw datas
- The raw data lists have equal length and all indexes are matched up

**take a look at what's in the dataframe**

In [58]:
toronto_post.head(10)

PostalCode           Borough                     Neighbourhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Harbourfront, Regent Park
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A      Queen's Park                      Not assigned
5        M9A         Etobicoke                  Islington Avenue
6        M1B       Scarborough                    Rouge, Malvern
7        M3B        North York                   Don Mills North
8        M4B         East York   Woodbine Gardens, Parkview Hill
9        M5B  Downtown Toronto          Ryerson, Garden District

**As per required, print the shape of the dataframe

In [59]:
toronto_post.shape

(103, 3)

## Part 2: Adding coordinates to the Dataframe

not reliable and too slow, not using geocoder

In [60]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('M5B'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

get coordinates from the given link

In [61]:
coord = pd.read_csv('http://cocl.us/Geospatial_data')
coord.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [62]:
coord.rename(columns={'Postal Code': 'PostalCode'},inplace=True)  # for consistency with toronto_post
coord.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

Merge the coord frame and toronto_post together

In [63]:
GTA = pd.merge(toronto_post, coord, how='left', on=['PostalCode'])
GTA.head(12)

PostalCode           Borough  \
0         M3A        North York   
1         M4A        North York   
2         M5A  Downtown Toronto   
3         M6A        North York   
4         M7A      Queen's Park   
5         M9A         Etobicoke   
6         M1B       Scarborough   
7         M3B        North York   
8         M4B         East York   
9         M5B  Downtown Toronto   
10        M6B        North York   
11        M9B         Etobicoke   

                                        Neighbourhood   Latitude  Longitude  
0                                           Parkwoods  43.753259 -79.329656  
1                                    Victoria Village  43.725882 -79.315572  
2                           Harbourfront, Regent Park  43.654260 -79.360636  
3                    Lawrence Heights, Lawrence Manor  43.718518 -79.464763  
4                                        Not assigned  43.662301 -79.389494  
5                                    Islington Avenue  43.667856 -79.532242  
6                                      Rouge, Malvern  43.806686 -79.194353  
7                                     Don Mills North  43.745906 -79.352188  
8                     Woodbine Gardens, Parkview Hill  43.706397 -79.309937  
9                            Ryerson, Garden District  43.657162 -79.378937  
10                                          Glencairn  43.709577 -79.445073  
11  Cloverdale, Islington, Martin Grove, Princess ...  43.650943 -79.554724

## Part 3: Explore the Neighborhood

get rows that borough contains "Toronto"

In [64]:
toronto = GTA[GTA['Borough'].str.contains('Toronto')]
toronto.head()

PostalCode           Borough              Neighbourhood   Latitude  \
2         M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
9         M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
15        M5C  Downtown Toronto             St. James Town  43.651494   
19        M4E      East Toronto                The Beaches  43.676357   
20        M5E  Downtown Toronto                Berczy Park  43.644771   

    Longitude  
2  -79.360636  
9  -79.378937  
15 -79.375418  
19 -79.293031  
20 -79.373306

In [65]:
toronto.shape

(38, 5)

In [66]:
toronto.reset_index(inplace=True, drop=True)  # reset index
toronto.head()

PostalCode           Borough              Neighbourhood   Latitude  \
0        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
2        M5C  Downtown Toronto             St. James Town  43.651494   
3        M4E      East Toronto                The Beaches  43.676357   
4        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306

search for toronto's coordinates

In [67]:
import geocoder
from geopy.geocoders import Nominatim

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


import a bunch of libraries

In [68]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


take a look at Toronto's map

In [69]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Using Foursquare API to explore the area 

hidden cell for my credentials

In [70]:
# @hidden_cell
CLIENT_ID = '4B04XQ1N4TVYTAWAGPRRYQITQGVWQALT1SOBPH3CSSSV0HNY' # your Foursquare ID
CLIENT_SECRET = '4HYWBMHSPGOS2CLEHPY4AWZDMGFJJNKKJM0E5JHPHXOANUHC' # your Foursquare Secret
VERSION = '20190519' # Foursquare API version

get some functiond from the tutorial

In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
toronto = toronto.rename(columns={"Neighbourhood":"Neighborhood"})
toronto.head(2)

PostalCode           Borough               Neighborhood   Latitude  \
0        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   

   Longitude  
0 -79.360636  
1 -79.378937

find venues using the above function

In [114]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [115]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                               50   
Berczy Park                                                            50   
Brockton, Exhibition Place, Parkdale Village                           20   
Business Reply Mail Processing Centre 969 Eastern                      17   
CN Tower, Bathurst Quay, Island airport, Harbou...                     15   
Cabbagetown, St. James Town                                            44   
Central Bay Street                                                     50   
Chinatown, Grange Park, Kensington Market                              50   
Christie                                                               16   
Church and Wellesley                                                   50   
Commerce Court, Victoria Hotel                                         50   
Davisville                                                             35   
Davisville North                                                        7   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     16   
Design Exchange, Toronto Dominion Centre                               50   
Dovercourt Village, Dufferin                                           18   
First Canadian Place, Underground city                                 50   
Forest Hill North, Forest Hill West                                     5   
Harbord, University of Toronto                                         36   
Harbourfront East, Toronto Islands, Union Station                      50   
Harbourfront, Regent Park                                              47   
High Park, The Junction South                                          25   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               50   
Moore Park, Summerhill East                                             3   
North Toronto West                                                     18   
Parkdale, Roncesvalles                                                 15   
Rosedale                                                                4   
Roselawn                                                                2   
Runnymede, Swansea                                                     36   
Ryerson, Garden District                                               50   
St. James Town                                                         50   
Stn A PO Boxes 25 The Esplanade                                        50   
Studio District                                                        37   
The Annex, North Midtown, Yorkville                                    24   
The Beaches                                                             5   
The Beaches West, India Bazaar                                         21   
The Danforth West, Riverdale                                           42   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                                50   
Berczy Park                                                             50   
Brockton, Exhibition Place, Parkdale Village                            20   
Business Reply Mail Processing Centre 969 Eastern                       17   
CN Tower, Bathurst Quay, Island airport, Harbou...                      15   
Cabbagetown, St. James Town                                             44   
Central Bay Street                                                      50   
Chinatown, Grange Park, Kensington Market                               50   
Christie                                                                16   
Church and Wellesley                                                    50   
Commerce Court, Victoria Hotel     

In [116]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 215 uniques categories.


Analyze each neighbourhood

In [117]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# group by neighbours
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [120]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond   American Restaurant   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village        Breakfast Spot   
3  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   
4  CN Tower, Bathurst Quay, Island airport, Harbo...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0           Coffee Shop                  Café            Steakhouse   
1          Cocktail Bar            Restaurant                  Café   
2           Coffee Shop                  Café          Climbing Gym   
3           Yoga Studio         Auto Workshop         Garden Center   
4        Airport Lounge      Airport Terminal                 Plane   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                   Bar      Asian Restaurant             Gastropub   
1        Farmers Market    Italian Restaurant            Steakhouse   
2    Italian Restaurant         Burrito Place                   Bar   
3                Garden  Fast Food Restaurant        Farmers Market   
4              Boutique               Airport    Airport Food Court   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Pizza Place                 Hotel         Breakfast Spot  
1              Beer Bar    Seafood Restaurant                 Bakery  
2  Caribbean Restaurant     Convenience Store                Stadium  
3            Comic Shop                  Park          Burrito Place  
4          Airport Gate      Sculpture Garden        Harbor / Marina

Cluster the neighborhoods

In [131]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough               Neighborhood   Latitude  \
0        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1        M5B  Downtown Toronto   Ryerson, Garden District  43.657162   
2        M5C  Downtown Toronto             St. James Town  43.651494   
3        M4E      East Toronto                The Beaches  43.676357   
4        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.360636               0           Coffee Shop                Bakery   
1 -79.378937               0                  Café           Coffee Shop   
2 -79.375418               0                  Café           Coffee Shop   
3 -79.293031               0              Boutique     Health Food Store   
4 -79.373306               0           Coffee Shop          Cocktail Bar   

       3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                        Pub                  Park                  Café   
1  Middle Eastern Restaurant               Theater        Cosmetics Shop   
2                 Restaurant             Gastropub          Cocktail Bar   
3                        Pub            Astrologer           Wings Joint   
4                 Restaurant                  Café        Farmers Market   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0               Theater    Mexican Restaurant            Restaurant   
1      Ramen Restaurant        Clothing Store              Beer Bar   
2        Farmers Market   Japanese Restaurant    Italian Restaurant   
3          Dance Studio   Dumpling Restaurant               Dog Run   
4    Italian Restaurant            Steakhouse              Beer Bar   

  9th Most Common Venue 10th Most Common Venue  
0        Breakfast Spot           Dessert Shop  
1                Bakery                  Hotel  
2                   Gym                  Hotel  
3              Dive Bar         Discount Store  
4    Seafood Restaurant                 Bakery

create a map

In [133]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

cluster 1

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0   Downtown Toronto               0           Coffee Shop   
1   Downtown Toronto               0                  Café   
2   Downtown Toronto               0                  Café   
3       East Toronto               0              Boutique   
4   Downtown Toronto               0           Coffee Shop   
5   Downtown Toronto               0           Coffee Shop   
6   Downtown Toronto               0         Grocery Store   
7   Downtown Toronto               0   American Restaurant   
8       West Toronto               0              Pharmacy   
9   Downtown Toronto               0           Coffee Shop   
10      West Toronto               0                   Bar   
11      East Toronto               0      Greek Restaurant   
12  Downtown Toronto               0           Coffee Shop   
13      West Toronto               0        Breakfast Spot   
15  Downtown Toronto               0           Coffee Shop   
16      East Toronto               0                  Café   
21      West Toronto               0                   Bar   
22   Central Toronto               0        Clothing Store   
23   Central Toronto               0                  Café   
24      West Toronto               0             Gift Shop   
25   Central Toronto               0          Dessert Shop   
26  Downtown Toronto               0                  Café   
27      West Toronto               0                  Café   
29  Downtown Toronto               0                  Café   
30   Central Toronto               0           Coffee Shop   
31  Downtown Toronto               0       Airport Service   
33  Downtown Toronto               0          Cocktail Bar   
34  Downtown Toronto               0           Coffee Shop   
35  Downtown Toronto               0                  Café   
36  Downtown Toronto               0           Coffee Shop   
37      East Toronto               0    Light Rail Station   

   2nd Most Common Venue          3rd Most Common Venue 4th Most Common Venue  \
0                 Bakery                            Pub                  Park   
1            Coffee Shop      Middle Eastern Restaurant               Theater   
2            Coffee Shop                     Restaurant             Gastropub   
3      Health Food Store                            Pub            Astrologer   
4           Cocktail Bar                     Restaurant                  Café   
5                   Café             Chinese Restaurant    Italian Restaurant   
6                   Café                           Park           Coffee Shop   
7            Coffee Shop                           Café            Steakhouse   
8                 Bakery                    Supermarket                  Park   
9                   Park                       Aquarium                  Café   
10           Coffee Shop               Asian Restaurant            Restaurant   
11           Coffee Shop             Italian Restaurant        Ice Cream Shop   
12                  Café                  Deli / Bodega            Restaurant   
13           Coffee Shop                           Café          Climbing Gym   
15                  Café                          Hotel         Deli / Bodega   
16           Coffee Shop                      Gastropub    Italian Restaurant   
21                  Café             Mexican Restaurant  Fast Food Restaurant   
22           Coffee Shop           Fast Food Restaurant    Chinese Restaurant   
23        Sandwich Place                    Coffee Shop           Pizza Place   
24        Breakfast Spot             Italian Restaurant      Cuban Restaurant   
25           Pizza Place                 Sandwich Place           Coffee Shop   
26                   Bar             Italian Restaurant   Japanese Restaurant   
27           Pizza Place             Italian Restaurant                   Gym   
29    Mexican Restaurant  Vegetarian / Vegan Restaurant                Bakery   
30 

cluster 2

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
28   Central Toronto               1            Playground   
32  Downtown Toronto               1                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
28                   Gym                  Park      Cuban Restaurant   
32            Playground                 Trail           Wings Joint   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
28               Dog Run              Dive Bar        Discount Store   
32      Cuban Restaurant               Dog Run              Dive Bar   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
28                 Diner    Dim Sum Restaurant           Dessert Shop  
32        Discount Store                 Diner     Dim Sum Restaurant

cluster 3

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
18  Central Toronto               2           Music Venue   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
18                Garden      Cuban Restaurant   Dumpling Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
18               Dog Run              Dive Bar        Discount Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
18                 Diner    Dim Sum Restaurant           Dessert Shop

cluster 4

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
17  Central Toronto               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
17    Dim Sum Restaurant              Bus Line           Swim School   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
17           Wings Joint          Cupcake Shop   Dumpling Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
17               Dog Run              Dive Bar         Discount Store

cluster 5

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
14     East Toronto               4                  Park   
19  Central Toronto               4     Food & Drink Shop   
20  Central Toronto               4                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14        Sandwich Place      Sushi Restaurant     Fish & Chips Shop   
19         Grocery Store                   Gym                  Park   
20                 Trail         Jewelry Store              Bus Line   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14            Board Shop               Brewery          Burger Joint   
19        Breakfast Spot        Sandwich Place                 Hotel   
20      Sushi Restaurant           Wings Joint          Cupcake Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
14         Burrito Place    Italian Restaurant                    Pub  
19           Wings Joint              Dive Bar         Discount Store  
20               Dog Run              Dive Bar         Discount Store